In [ ]:
# importing required modules 
from zipfile import ZipFile 
  
# specifying the zip file name 
zipfile = '../pentaho/divvy/divvy_data/Divvy_Trips_2016_Q3Q4.zip' 
  
with ZipFile(zipfile, 'r') as zip: 
    # printing all the contents of the zip file
    files = zip.namelist()
print(files)

In [ ]:
import re
r = re.compile("Divvy_Trips_.*Divvy_Trips.*csv$")
csvlist = list(filter(r.match, files))
print(csvlist)
    

In [ ]:
import pandas as pd
import numpy as np

df = pd.concat([pd.read_csv(f) for f in csvlist])
df

In [ ]:
import requests
import json
from pandas.io.json import json_normalize

In [ ]:
r = requests.get('https://feeds.divvybikes.com/stations/stations.json')
if r.status_code == 200:
    live_station_data = json.loads(r.text)
    print("live data received")
else:
    print("could not get live station data")

In [ ]:
station_data_raw  = json_normalize(live_station_data['stationBeanList'])
station_data = station_data_raw[['id','latitude','longitude','status','stationName','availableDocks','totalDocks','availableBikes']]
station_data

In [ ]:
from_trips = pd.merge(df, station_data, left_on='from_station_id', right_on='id')
to_trips = pd.merge(df, station_data, left_on='to_station_id', right_on='id')

In [ ]:
#grouped = from_trips.groupby('from_station_id')
#from_trips.groupby(['from_station_name']).count()
#df[['col1', 'col2', 'col3', 'col4']].groupby(['col1', 'col2']).agg(['mean', 'count'])
#from_trips.groupby(['from_station_id']).agg(['count'])
#from_trips.groupby(['from_station_id']).size().reset_index(name='counts')
from_count_total = from_trips.groupby(['from_station_id'])['from_station_id'].count().reset_index(name='from_count')
to_count_total = from_trips.groupby(['to_station_id'])['to_station_id'].count().reset_index(name='to_count')

In [ ]:
#verifying counts above
from_trips[from_trips['from_station_id']==562]
#to_trips[to_trips['to_station_id']==562]


In [ ]:
from_trips_counts = pd.merge(from_trips, from_count_total, on='from_station_id')
to_trips_counts = pd.merge(to_trips, to_count_total, on='to_station_id')

df_to_counts = to_trips_counts[['to_station_name','latitude','longitude','availableDocks','totalDocks','availableBikes','to_count']].sort_values(by='to_count')
df_from_counts = from_trips_counts[['from_station_name','latitude','longitude','availableDocks','totalDocks','availableBikes','from_count']].sort_values(by='from_count')

In [ ]:
def name_range(row):
    if row['counts'] < 10:
        range_size = 'XS'
    elif row['counts'] <= 10 and row['counts'] < 100:
        range_size = 'S'
    elif row['counts'] <= 100 and row['counts'] < 1500:
        range_size = 'M'
    elif row['counts'] <= 1500 and row['counts'] < 5000:
        range_size = 'L'
    elif row['counts'] <= 1500 and row['counts'] < 5000:
        range_size = 'XL'
    else:
        range_size = 'unknown'
    return range_size

In [ ]:
#df_counts['range'] = df_counts.apply(name_range, axis=1)
#df_counts.count()
df_from_final = df_from_counts.drop_duplicates()
df_to_final = df_to_counts.drop_duplicates()

In [ ]:
df_from_final.loc[df_from_final['from_count'] < 10, 'from_range'] = 'XS'
df_from_final.loc[(df_from_final['from_count'] >= 10) & (df_from_final['from_count'] < 100), 'from_range'] = 'S'
df_from_final.loc[(df_from_final['from_count'] >= 100) & (df_from_final['from_count'] < 1500), 'from_range'] = 'M'
df_from_final.loc[(df_from_final['from_count'] >= 1500) & (df_from_final['from_count'] < 5000), 'from_range'] = 'L'
df_from_final.loc[df_from_final['from_count'] > 5000, 'from_range'] = 'XL'

df_to_final.loc[df_to_final['to_count'] < 10, 'to_range'] = 'XS'
df_to_final.loc[(df_to_final['to_count'] >= 10) & (df_to_final['to_count'] < 100), 'to_range'] = 'S'
df_to_final.loc[(df_to_final['to_count'] >= 100) & (df_to_final['to_count'] < 1500), 'to_range'] = 'M'
df_to_final.loc[(df_to_final['to_count'] >= 1500) & (df_to_final['to_count'] < 5000), 'to_range'] = 'L'
df_to_final.loc[df_to_final['to_count'] > 5000, 'to_range'] = 'XL'


In [ ]:
df_display = pd.merge(df_to_final, df_from_final, left_on='to_station_name', right_on='from_station_name')
df_display_final = df_display[['from_station_name','latitude_x','longitude_x','availableDocks_x','totalDocks_x','availableBikes_x','to_count','to_range','from_count','from_range']]

df_display_final.rename(columns={'from_station_name': 'stationName', 'latitude_x': 'latitude','longitude_x':'longitude','availableDocks_x'\
               :'availableDocks','totalDocks_x':'totalDocks','availableBikes_x':'availableBikes'}, inplace=True)
df_display_final